In [1]:
import os
from unstructured.partition.pdf import partition_pdf


# Get the current working directory
current_working_directory = os.getcwd()
# Print the current working directory
print(current_working_directory)

/Users/hudanyunsheng/miniforge3/envs/jpfaq/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/hudanyunsheng/Documents/GitHub/JP_FAQ/notebooks


```
conda install conda-forge::poppler  
conda install conda-forge::tesseract  
```

On Mac:  
`brew install tesseract`  
`brew install poppler-utils`

In [3]:
# from unstructured_inference.models.tables import cells_to_html

In [109]:
# path_pdf = "../data/JP Label/stelara iv Japanese PI.pdf"
path_pdf = "../data/JP Label/stelara sc Japanese PI.pdf"
filename = path_pdf.split("/")[-1].split(".pdf")[0]
dir_partitions = "../output/JP Label images partition/"
if not os.path.exists(dir_partitions):
    os.makedirs(dir_partitions)

In [110]:
raw_pdf_elements = partition_pdf(
    filename=path_pdf,                  # mandatory
    strategy="hi_res",                                     # mandatory to use ``hi_res`` strategy
    extract_images_in_pdf=True,                            # mandatory to set as ``True``
    extract_image_block_types=["Image", "Table"],          # optional
    extract_image_block_to_payload=False,                  # optional
    extract_image_block_output_dir=dir_partitions + filename, #.split(".")[0] + "/",  # optional - only works when 
    chunking_strategy="by_title",
    # extract_image_block_to_payload=True
    multipage_sections = "False",
    include_page_breaks = "False",
    )

In [111]:
print(f"In total {len(raw_pdf_elements)} elements")
ele_types = set([ele.category for ele in raw_pdf_elements])
print(f"Categories of all elements are: {ele_types}")

In total 42 elements
Categories of all elements are: {'CompositeElement', 'Table'}


In [113]:
ele_groups = dict((key, []) for key in ele_types)

for ele in raw_pdf_elements:
    ele_groups[ele.category].append(ele)
print(f"{len(ele_groups['Table'])} tables")

7 tables


In [114]:
info_tables = [x.to_dict() for x in ele_groups['Table']]
info_images = [x.to_dict() for x in ele_groups['CompositeElement']]

In [115]:
from collections import defaultdict

info_tables_page = defaultdict(list)
for info_table in info_tables:
    pg = info_table['metadata']['page_number']
    info_tables_page[f"page {pg}"].append(info_table)

info_images_page = defaultdict(list)
for info_image in info_images:
    pg = info_table['metadata']['page_number']
    info_images_page[f"page {pg}"].append(info_image)

In [116]:
info_tables_page["page 1"][0]['text']

'販売名 ステラーラ皮下注45mgシリンジ 有効成分 （1シリンジ0.5mL中） ウステキヌマブ（遺伝子組換え）45mg含有 添加剤 精製白糖38mg、L-ヒスチジン0.5mg、ポリソ ルベート80 0.02mg'

In [117]:
import re
import pymupdf

In [118]:
texts = []
doc = pymupdf.open(path_pdf) # open a document
for page in doc: # iterate the document pages
    text = page.get_text() # get plain text encoded as UTF-8
    texts.append(text)

In [119]:
tbs = {}
bboxes = {}
n = 1
for page in doc:
    tbs[f"page {n}"] = []
    bboxes[f"page {n}"] = []
    print(f"Page {n}:")
    n_tb = 0
    for tb in page.find_tables(vertical_strategy="lines_strict", horizontal_strategy="lines_strict", snap_x_tolerance=1): 
        # print(len(tb.cells))
        n_tb += 1
        tbs[f"page {n}"].append(tb.extract())
        bboxes[f"page {n}"].append(tb.bbox) # x0, y0, x1, y1
    n += 1
    print(f"{n_tb} tables")

Page 1:
4 tables
Page 2:
1 tables
Page 3:
2 tables
Page 4:
2 tables


In [120]:
tbs['page 1'][2]

[['販売名', 'ステラーラ皮下注45mgシリンジ'],
 ['有効成分', '（1シリンジ0.5mL中）\nウステキヌマブ（遺伝子組換え）45mg含有'],
 ['添加剤', '精製白糖38mg、L-ヒスチジン0.5mg、ポリソ\nルベート80 0.02mg']]

In [121]:
text = "\n".join(texts)
result = {}
current_key = ''
lines = text.splitlines()
for line in lines:
    section_match = re.match(r'^(\d+\.\s+.+)', line)
    if section_match:
        # current_key = line.split(' ', 1)[1]
        current_key = section_match.group(1).strip()
        result[current_key] = []
    else:
        if current_key in result:
            result[current_key].append(line.strip())

In [122]:
tbs["page 1"]

[[['貯 法：2～8℃で保存すること\n有効期間：36ヵ月', '日本標準商品分類番号'], [None, '873999']],
 [['Stelara Subcutaneous Injection\n※注意－医師等の処方箋により使用すること',
   '承認番号',
   '22300AMX00422000'],
  [None, '販売開始', '2011年3月']],
 [['販売名', 'ステラーラ皮下注45mgシリンジ'],
  ['有効成分', '（1シリンジ0.5mL中）\nウステキヌマブ（遺伝子組換え）45mg含有'],
  ['添加剤', '精製白糖38mg、L-ヒスチジン0.5mg、ポリソ\nルベート80 0.02mg']],
 [['色・性状', '無色～淡黄色の澄明又はわずかに混濁した液'],
  ['pH', '5.7～6.3'],
  ['浸透圧比', '約1（生理食塩液に対する比）']]]

In [123]:
# def parse_format_1(data):
#     # Extract the main sections
#     main_parts = data.split(' 投与量 ')
#     weight_part = main_parts[0].replace('患者体重 ', '')
#     dose_part = main_parts[1]

#     # Split the weight and dose parts
#     weights = re.split(r' (?=\d+kg)', weight_part)
#     doses = dose_part.split(' ')

#     # Combine into the structured format
#     structured_format = [['患者体重', '投与量']]
#     for weight, dose in zip(weights, doses):
#         structured_format.append([weight, dose])

#     return structured_format
    
# parse_format_1(info_tables_page[1][0]['text'])

In [124]:
# def convert_format_2_to_string(data):
#     # Combine header
#     header = f"{data[0][0]} {data[0][1]}"
    
#     # Combine the rest of the data
#     body = ' '.join([f"{item[0]} {item[1]}" for item in data[1:]])
    
#     # Combine header and body into one string
#     return f"{header} {body}"

# convert_format_2_to_string(tbs['page 1'][2])

In [127]:
info_tables_page["page 1"][0]['text']

'販売名 ステラーラ皮下注45mgシリンジ 有効成分 （1シリンジ0.5mL中） ウステキヌマブ（遺伝子組換え）45mg含有 添加剤 精製白糖38mg、L-ヒスチジン0.5mg、ポリソ ルベート80 0.02mg'

In [129]:
import difflib
def compare_strings_similarity(str1, str2):
    return difflib.SequenceMatcher(None, str1, str2).ratio()

compare_strings_similarity(info_tables_page["page 1"][0]['text'], convert_format_2_to_string(tbs['page 1'][2]))

0.9809523809523809

In [130]:
compare_strings_similarity(info_tables_page["page 1"][0]['text'], ' '.join([item for sublist in tbs['page 1'][2] for item in sublist]))

0.9809523809523809

In [131]:
scores = dict()
for pg, tbs_p in tbs.items():
    info_tbs_p = info_tables_page[pg]
    scores[pg] = dict()

    # one-to-one comparison
    for i in range(len(info_tbs_p)):
        info_tb = info_tbs_p[i]
        scores[pg][i] = []#dict()
        
        for j in range(len(tbs_p)):
            tb_p = tbs_p[j]           
            # scores[pg][i][j] = compare_strings_similarity(info_tb['text'], ' '.join([item for sublist in tb_p for item in sublist if item is not None]))
            scores[pg][i].append(compare_strings_similarity(info_tb['text'], ' '.join([item for sublist in tb_p for item in sublist if item is not None])))
scores

{'page 1': {0: [0.08108108108108109,
   0.05235602094240838,
   0.9809523809523809,
   0.07453416149068323],
  1: [0.08080808080808081, 0.07042253521126761, 0.07453416149068323, 1.0]},
 'page 2': {0: [0.8560460652591171]},
 'page 3': {0: [0.4217391304347826, 0.2057877813504823],
  1: [0.026578073089700997, 0.9605263157894737]},
 'page 4': {0: [0.8247422680412371, 0.582010582010582],
  1: [0.6666666666666666, 0.9519230769230769]}}

In [132]:
import numpy as np
mapping = dict()
for pg, score in scores.items():
    print(pg)
    info_tbs_p = info_tables_page[pg]
    tbs_p = tbs[pg]
    mapping[pg] = dict()
    for i in range(len(info_tbs_p)):
        # print(scores[pg][i], np.argmax(scores[pg][i]))
        print(f"\t{info_tbs_p[i]}")
        print(f"{tbs_p[np.argmax(score[i])]}")
        mapping[pg][i] = np.argmax(score[i])

page 1
	{'type': 'Table', 'element_id': '6add13b77fa2ee0f4f10a3b26921f094', 'text': '販売名 ステラーラ皮下注45mgシリンジ 有効成分 （1シリンジ0.5mL中） ウステキヌマブ（遺伝子組換え）45mg含有 添加剤 精製白糖38mg、L-ヒスチジン0.5mg、ポリソ ルベート80 0.02mg', 'metadata': {'last_modified': '2024-07-30T16:01:57', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'orig_elements': 'eJx1Uttu2zAM/RVDz7Ej2bo5f7AiAwKsb0kQ6EInHnxDLGMNiv77RLndiqF7IeijQ/IcmsdXAh30MIRL68kuI5WQlXYNz7lTVc4BWK4riJ/KlpTWupZKkE1GegjGm2BizStx43j37WACzOm7M49xCZcbtNdbiEhZVXWseYd/tT7cIsqk4BGdxnYIWHc8MiELscmYYLyoz5vsDyBrUWgEVEzkV8BaEhEyP+YAPXo5tC/Q/ZiMA/IWHzwEcKEdh4vrzDxfpvtoI40WQqoSlTRtB+ExQao9fEebbW+ucJlMEkyKYhsNTEvYPh2yvbHQZYkwZ5O5hxZ7b+PwztxNNrvsyUxmgBmyw7dtMLaDnOWs+DldSVrGcF2wFp0TGK7knNA5XPrRt00L6X+UtOQ5VXlFn5ncUbYTCqsnlDUsvYU7bhLtBXjBXZPTop2pT4vQTRkjpz47LRW1NUYnMULKG/eRK2ngtHCg9rRIB5qL/ppqVGKYxK4SomMzqSj2L7GlLBnFnMr40DRUs/8Vxj33e5xS+sSskywj/xVnfMqTOA8pyrX1aampij15wyJHWBEnK895VFF5i0yu19arAcFLu6pF0b6xKNQkuZJHSDkb+2vdeFyBwuh8PP/VPGX7PE0vPylc3enVFzp6pyai/2varkuHpMp/W

In [133]:
mapping

{'page 1': {0: 2, 1: 3},
 'page 2': {0: 0},
 'page 3': {0: 0, 1: 1},
 'page 4': {0: 0, 1: 1}}